In [3]:
import sys
sys.path.append("{workfolder}/Desktop/Uni/GraphToSequenceNN/Scripts/")
import numpy as np
import tensorflow as tf
from keras import backend as K
from keras.layers import multiply
from keras.models import Model
from keras.callbacks import History
from NetworkHandler.KerasSupportMethods.SupportMethods import KerasEval as KE
from NetworkHandler.KerasSupportMethods.SupportMethods import AssertTensorDotDim, AssertTensorShapeEqual, AssertNotNone, AssertNotNegative
from NetworkHandler.KerasSupportMethods.SupportMethods import AssertAddTensorToTensor, ConsoleReporter, AssertIsTensor
from NetworkHandler.KerasSupportMethods.ControledBasicOperations import ControledTensorOperations as CTO
from NetworkHandler.Neighbouring.NeighbourhoodCollector import Neighbourhood as Nhood

#These are the edges between the nodes.
current = np.array( [
                    [0,0.3,0.3,1,0.3],
                    [0.1,0.3,1,0,0.2],
                    [0,0.7,1,1,0.4],
                    [0.8,0.2,1,1,0.2]
                    ], dtype='float32')

current_ten = K.variable (value=current, dtype='float32')

################################

neigbours = np.array([
                     [0,1,1,1],
                     [0,0,0,1],
                     [0,0,0,0],
                     [0,0,0,0]
                     ], dtype='float32')

neighbouring = K.variable (value=neigbours, dtype='float32')

zero_vec = K.zeros(current[0,:].shape, dtype='float32')


################################
'''
weights = np.array([
                   [0, 0.3, 0.1, 1, 0.3],
                   [1, 0.1, 0.1, 1, 0.9], 
                   [0, 0.3, 0.1, 0.9, 0.3],
                   [0.6, 0.3, 1, 0.7, 0.1]
                   ], dtype='float32')
'''
concate_multiplier = 2
in_dim = current_ten.shape[1] + neighbouring.shape[1]
out_dim = 100

weight_array = np.random.rand(in_dim,out_dim)
weights_ten = K.variable (value=weight_array, dtype='float32')

bias_array = np.zeros(out_dim)
bias_ten = K.variable (value=bias_array, dtype='float32')

sess = tf.Session()
K.set_session(sess)
init_g = tf.global_variables_initializer()
init_l = tf.local_variables_initializer()
with sess:
    init_g.run()
    init_l.run()
    
    
    
    ConsoleReporter('Inits',current_ten,True)
    ConsoleReporter('Neighbouring',neighbouring,True)
    ConsoleReporter('weights_ten',weights_ten,False)
    
    agg_neigh_vecs = Nhood(current_ten, neighbouring).Execute()
    ConsoleReporter('agg_neigh_vecs',agg_neigh_vecs,True)
    
    concatenated = CTO.ControledConcatenation(current_ten, agg_neigh_vecs)
    ConsoleReporter('concatenated',concatenated,True)
    
    multiplied = CTO.ControledWeightDotProduct(concatenated, weights_ten)
    ConsoleReporter('multiplied',multiplied,False)
    
    biased = CTO.ControledBiased(multiplied, bias_ten)
    ConsoleReporter('biased',biased,False)


Inits :
 [[0.  0.3 0.3 1.  0.3]
 [0.1 0.3 1.  0.  0.2]
 [0.  0.7 1.  1.  0.4]
 [0.8 0.2 1.  1.  0.2]] 

Neighbouring :
 [[0. 1. 1. 1.]
 [0. 0. 0. 1.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]] 

weights_ten :
 <tf.Variable 'Variable_9:0' shape=(9, 100) dtype=float32_ref> 

agg_neigh_vecs :
 [[0.   0.   0.   0.  ]
 [0.32 0.   0.   0.  ]
 [0.62 0.   0.   0.  ]
 [0.64 0.64 0.   0.  ]] 

concatenated :
 [[0.   0.3  0.3  1.   0.3  0.   0.   0.   0.  ]
 [0.1  0.3  1.   0.   0.2  0.32 0.   0.   0.  ]
 [0.   0.7  1.   1.   0.4  0.62 0.   0.   0.  ]
 [0.8  0.2  1.   1.   0.2  0.64 0.64 0.   0.  ]] 

multiplied :
 Tensor("MatMul_1:0", shape=(4, 100), dtype=float32) 

biased :
 Tensor("add_1:0", shape=(4, 100), dtype=float32) 

